In [ ]:
import os
from openslide import OpenSlide

In [ ]:
pth = r'C:\Users\kyuha\Desktop\downloadfromlab'
items = [_ for _ in os.listdir(pth) if _.endswith('svs')]
item = items[0]
svs = OpenSlide(os.path.join(pth,item))

In [ ]:
svs.level_count
svs.dimensions
svs.level_dimensions
svs.level_downsamples
svs.properties
svs.associated_images

In [ ]:
svs.associated_images['label']
svs.associated_images['thumbnail']

In [ ]:
svs.level_dimensions
# level 0,1,2,3

In [ ]:
top = 0
left = 0
level = 1
size = (6000,6000)
location = (left,top)
region = svs.read_region(location, level, size)

In [ ]:
region

In [5]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os
pth = r'C:\Users\kyuha\Desktop\downloadfromlab'
xmls = [_ for _ in os.listdir(pth) if _.endswith('xml')]
xml = xmls[0]
xml

'5619_Wirtz.Denis_OTS-19_5619-018.xml'

In [8]:
tree = ET.parse(os.path.join(pth,xml))
root = tree.getroot()
# pre-allocate arrays of interest
x = np.array([])
y = np.array([])
obj = np.array([])
label = np.array([])

## CSV/ pandas dataframe structure
1st column = tissue type/class (1~14)
2nd column = annotation ID (1~nn) / # of circles
3rd column = x-coordinate
4th column = y-coordinate

1   1   x1  y1
1   1   x2  y2

If you want to crop svs with 1st class, 1st annotation circle,
you can index 0 in 1st column and 0 in 2nd column, get all x,y coordinate
find min and max of x and y to get bounding box. and crop using read_region.

In [ ]:
for Annotation in root.iter('Annotation'):
  for Region in Annotation.iter('Region'):
     xx = np.array([int(Vertex.get('X')) for Vertex in Region.iter('Vertex')])
     yy = np.array([int(Vertex.get('Y')) for Vertex in Region.iter('Vertex')])
     objj = np.array([int(Region.get('Id'))]*len(xx))
     labell = np.array([int(Annotation.get('Id'))]*len(xx))
     x = np.concatenate((x,xx),axis=None)
     y = np.concatenate((y,yy),axis=None)
     obj = np.concatenate((obj,objj),axis=None)
     label = np.concatenate((label,labell),axis=None)
print('number of coordinates in annotation : ',len(x))
mdict = {'x':x,'y':y,'objID':obj,'label':label}
df = pd.DataFrame(mdict)
df.to_csv(index_label=['',''],header=['','',''])

create new dataframe with bounding box coordinates

Example Dataframe structure for list of bounding boxes

classID objectID    minx    miny    maxx    maxy
    1       1       minx1  miny1    maxx1   maxy1
    1       2       minx1  miny1    maxx2   maxy2

In [ ]:
import pillow
for bounding_box in df2:
    top = bounding_box['miny']
    left = bounding_box['minx']
    level = 1
    size = (maxx-minx,maxy-miny)
    location = (left,top)
    region = svs.read_region(location, level, size)
    region.save('filenameeijdvnjs.jpeg',"JPEG")

read the saved bounding boxes and crop into fixed size tiles (224x224)
tile size depends on your CNN model.

you could crop bounding boxes before saving it. try to crop into tiles only on the
annotated region in the bounding boxes and save tiles.

label class and object ID into filename or save them into subfolders with labels.